In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from linearmodels.panel import PanelOLS, RandomEffects, PooledOLS
from linearmodels.panel import compare
from scipy import stats
import numpy.linalg as la
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [3]:
df = pd.read_csv("final.csv", sep=';')
df['Data'] = pd.to_datetime(df['Data'])
df = df.set_index(['Store_ID', 'Data'])
# print(df.head())
# print(df.columns)

In [4]:
string_to_float_columns = [
    'Area', 'Gross_Margin_Percentage_Store', 'Gross_Margin_Store',
    'Number_of_Food_Service_Units_Sold', 'Number_of_Beverages_Units_Sold',
    'Turnover_Food_Service', 'Turnover_Beverages', 'Share_in_Total_Sales_Food_Service',
    'Number_of_Receipts_Food_Service', 'Number_of_Receipts_Beverages',
    'Gross_Profit_Margin_Food_Service', 'Gross_Profit_Margin_Beverages',
    'Share_of_Food_Service_Margin_in_Total_Margin', 'Discount_Amount_Food_Service',
    'Discount_Amount_Beverages', 'Inventory_on_Market_Food_Service',
    'Inventory_on_Market_Beverages', 'Inventory_Balance_Food_Service',
    'Inventory_Balance_Beverages', 'Inventory_Turnover_Ratio_Food_Service',
    'Inventory_Turnover_Ratio_', 'GMROI_Food_Service', 'GMROI_Beverages',
    'Losses_Beverages'
]

for col in string_to_float_columns:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace('Е', 'E', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float)
    )
df[string_to_float_columns].dtypes

Area                                            float64
Gross_Margin_Percentage_Store                   float64
Gross_Margin_Store                              float64
Number_of_Food_Service_Units_Sold               float64
Number_of_Beverages_Units_Sold                  float64
Turnover_Food_Service                           float64
Turnover_Beverages                              float64
Share_in_Total_Sales_Food_Service               float64
Number_of_Receipts_Food_Service                 float64
Number_of_Receipts_Beverages                    float64
Gross_Profit_Margin_Food_Service                float64
Gross_Profit_Margin_Beverages                   float64
Share_of_Food_Service_Margin_in_Total_Margin    float64
Discount_Amount_Food_Service                    float64
Discount_Amount_Beverages                       float64
Inventory_on_Market_Food_Service                float64
Inventory_on_Market_Beverages                   float64
Inventory_Balance_Food_Service                  

In [5]:
log_candidates = [
    'Number_of_Food_Service_Units_Sold',
    'Number_of_Beverages_Units_Sold',

    'Discount_Amount_Food_Service',
    'Discount_Amount_Beverages',

    'Inventory_Turnover_Ratio_Food_Service',
    'Inventory_Turnover_Ratio_',

    'Losses_Beverages'
]

skewness = df[log_candidates].skew()
print(skewness)

Number_of_Food_Service_Units_Sold        3.383530
Number_of_Beverages_Units_Sold           2.163696
Discount_Amount_Food_Service             2.183953
Discount_Amount_Beverages                7.256808
Inventory_Turnover_Ratio_Food_Service    2.448384
Inventory_Turnover_Ratio_                1.660443
Losses_Beverages                         2.560935
dtype: float64


In [6]:
for col in log_candidates:
    if (df[col] <= 0).any():
        df[f'log_{col}'] = np.log1p(df[col])  # log(1 + x)
    else:
        df[f'log_{col}'] = np.log(df[col])

log_cols = [f'log_{col}' for col in log_candidates]
print("\nСкошеність після логарифмування:")
print(df[log_cols].skew())


Скошеність після логарифмування:
log_Number_of_Food_Service_Units_Sold       -0.351980
log_Number_of_Beverages_Units_Sold          -3.109046
log_Discount_Amount_Food_Service            -0.769320
log_Discount_Amount_Beverages               -0.158086
log_Inventory_Turnover_Ratio_Food_Service   -2.963709
log_Inventory_Turnover_Ratio_               -7.402679
log_Losses_Beverages                        -1.250348
dtype: float64


c:\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
fe_vars_without_log = [
    'Number_of_Food_Service_Units_Sold',

    'Discount_Availability_Food_Service',
    'Discount_Amount_Food_Service',
    'Discount_Availability_Bevareges',
    'Discount_Amount_Beverages',

    'Inventory_Turnover_Ratio_Food_Service',
    'Inventory_Turnover_Ratio_',

    '2022_Dummy',
    '2023_Dummy',

    'February_Dummy', 'Murch_Dummy', 'April_Dummy', 'May_Dummy',
    'June_Dummy', 'July_Dummy', 'August_Dummy', 'September_Dummy',
    'October_Dummy', 'November_Dummy', 'December_Dummy',

    'Vacation_Time',
    'Competitor_Presence',
    'Office_Dummy',
    'School_Dummy', 
    'Type_Store'
]

fe_vars = [
    'Number_of_Food_Service_Units_Sold',

    'Discount_Availability_Food_Service',
    'log_Discount_Amount_Food_Service',
    'Discount_Availability_Bevareges',
    'log_Discount_Amount_Beverages',

    'log_Inventory_Turnover_Ratio_Food_Service',
    'Inventory_Turnover_Ratio_',

    '2022_Dummy',
    '2023_Dummy',

    'February_Dummy', 'Murch_Dummy', 'April_Dummy', 'May_Dummy',
    'June_Dummy', 'July_Dummy', 'August_Dummy', 'September_Dummy',
    'October_Dummy', 'November_Dummy', 'December_Dummy',

    'Vacation_Time',
    'Competitor_Presence',
    'Office_Dummy',
    'School_Dummy',
    'Type_Store'
]

In [8]:
X = df[fe_vars_without_log].dropna()
X = add_constant(X) 
vif_data = pd.DataFrame()
vif_data['Variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(
    X.values, i) for i in range(X.shape[1])]

print(vif_data.sort_values(by='VIF', ascending=False))

                                 Variable       VIF
11                            Murch_Dummy       inf
10                         February_Dummy       inf
18                          October_Dummy       inf
19                         November_Dummy       inf
21                          Vacation_Time       inf
17                        September_Dummy       inf
13                              May_Dummy       inf
12                            April_Dummy       inf
9                              2023_Dummy  2.134285
20                         December_Dummy  2.055756
16                           August_Dummy  2.029046
14                             June_Dummy  2.019328
1       Number_of_Food_Service_Units_Sold  1.997717
15                             July_Dummy  1.934435
8                              2022_Dummy  1.815844
7               Inventory_Turnover_Ratio_  1.643118
3            Discount_Amount_Food_Service  1.578020
25                             Type_Store  1.421754
6   Inventor

c:\Python312\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
c:\Python312\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


In [9]:
fe_model = PanelOLS.from_formula(
    'Number_of_Beverages_Units_Sold ~ Number_of_Food_Service_Units_Sold + Discount_Availability_Food_Service + Discount_Amount_Food_Service + Discount_Availability_Bevareges + Discount_Amount_Beverages + Inventory_Turnover_Ratio_Food_Service + Inventory_Turnover_Ratio_ + 2022_Dummy + 2023_Dummy + February_Dummy + Murch_Dummy + April_Dummy + May_Dummy + June_Dummy + July_Dummy + August_Dummy + September_Dummy + October_Dummy + November_Dummy + December_Dummy',
    data=df)
fe_results = fe_model.fit(cov_type='robust')
print(str(fe_results.summary).replace("PanelOLS", "Fixed Effects"))

                                Fixed Effects Estimation Summary                                 
Dep. Variable:     Number_of_Beverages_Units_Sold   R-squared:                        0.9435
Estimator:                               Fixed Effects   R-squared (Between):              0.9582
No. Observations:                            2470   R-squared (Within):               0.6396
Date:                            Sun, May 04 2025   R-squared (Overall):              0.9435
Time:                                    23:37:41   Log-likelihood                -2.149e+04
Cov. Estimator:                            Robust                                           
                                                    F-statistic:                      2045.0
Entities:                                      88   P-value                           0.0000
Avg Obs:                                   28.068   Distribution:                 F(20,2450)
Min Obs:                                   1.0000           

In [10]:
fe_model_log = PanelOLS.from_formula(
    'Number_of_Beverages_Units_Sold ~ Number_of_Food_Service_Units_Sold + Discount_Availability_Food_Service + log_Discount_Amount_Food_Service + Discount_Availability_Bevareges + log_Discount_Amount_Beverages + log_Inventory_Turnover_Ratio_Food_Service + Inventory_Turnover_Ratio_ + 2022_Dummy + 2023_Dummy + February_Dummy + Murch_Dummy + April_Dummy + May_Dummy + June_Dummy + July_Dummy + August_Dummy + September_Dummy + October_Dummy + November_Dummy + December_Dummy',
    data=df)
fe_results_log = fe_model_log.fit(cov_type='robust')

print(str(fe_results_log.summary).replace("PanelOLS", "Fixed Effects"))

                                Fixed Effects Estimation Summary                                 
Dep. Variable:     Number_of_Beverages_Units_Sold   R-squared:                        0.9427
Estimator:                               Fixed Effects   R-squared (Between):              0.9564
No. Observations:                            2470   R-squared (Within):               0.6382
Date:                            Sun, May 04 2025   R-squared (Overall):              0.9427
Time:                                    23:37:41   Log-likelihood                -2.151e+04
Cov. Estimator:                            Robust                                           
                                                    F-statistic:                      2015.2
Entities:                                      88   P-value                           0.0000
Avg Obs:                                   28.068   Distribution:                 F(20,2450)
Min Obs:                                   1.0000           

In [11]:
re_model = RandomEffects.from_formula(
    'Number_of_Beverages_Units_Sold ~ Number_of_Food_Service_Units_Sold + '
    'Discount_Availability_Food_Service + Discount_Amount_Food_Service + '
    'Discount_Availability_Bevareges + Discount_Amount_Beverages + '
    'Inventory_Turnover_Ratio_Food_Service + Inventory_Turnover_Ratio_ + '
    '2022_Dummy + 2023_Dummy + '
    'February_Dummy + Murch_Dummy + April_Dummy + May_Dummy + June_Dummy + '
    'July_Dummy + August_Dummy + September_Dummy + October_Dummy + '
    'November_Dummy + December_Dummy + '
    'Vacation_Time + Competitor_Presence + Office_Dummy + School_Dummy + '
    'Type_Store',
    data=df
)
re_results = re_model.fit(cov_type='robust')
print(re_results.summary)

                              RandomEffects Estimation Summary                              
Dep. Variable:     Number_of_Beverages_Units_Sold   R-squared:                        0.6729
Estimator:                          RandomEffects   R-squared (Between):              0.6792
No. Observations:                            2470   R-squared (Within):               0.6700
Date:                            Sun, May 04 2025   R-squared (Overall):              0.7042
Time:                                    23:37:42   Log-likelihood                -2.067e+04
Cov. Estimator:                            Robust                                           
                                                    F-statistic:                      209.61
Entities:                                      88   P-value                           0.0000
Avg Obs:                                   28.068   Distribution:                 F(24,2445)
Min Obs:                                   1.0000                     

In [12]:
re_model_log = RandomEffects.from_formula(
    'Number_of_Beverages_Units_Sold ~ Number_of_Food_Service_Units_Sold + '
    'Discount_Availability_Food_Service + log_Discount_Amount_Food_Service + '
    'Discount_Availability_Bevareges + log_Discount_Amount_Beverages + '
    'log_Inventory_Turnover_Ratio_Food_Service + Inventory_Turnover_Ratio_ + '
    '2022_Dummy + 2023_Dummy + '
    'February_Dummy + Murch_Dummy + April_Dummy + May_Dummy + June_Dummy + '
    'July_Dummy + August_Dummy + September_Dummy + October_Dummy + '
    'November_Dummy + December_Dummy + '
    'Vacation_Time + Competitor_Presence + Office_Dummy + School_Dummy + '
    'Type_Store',
    data=df
)
re_results_log = re_model_log.fit(cov_type='robust')
print(re_results_log.summary)

                              RandomEffects Estimation Summary                              
Dep. Variable:     Number_of_Beverages_Units_Sold   R-squared:                        0.6722
Estimator:                          RandomEffects   R-squared (Between):              0.6747
No. Observations:                            2470   R-squared (Within):               0.6698
Date:                            Sun, May 04 2025   R-squared (Overall):              0.7020
Time:                                    23:37:42   Log-likelihood                -2.067e+04
Cov. Estimator:                            Robust                                           
                                                    F-statistic:                      208.91
Entities:                                      88   P-value                           0.0000
Avg Obs:                                   28.068   Distribution:                 F(24,2445)
Min Obs:                                   1.0000                     

In [13]:
def hausman(fe, re):
    in_both_coef = fe.params.index.intersection(re.params.index)
    b = fe.params[in_both_coef]
    B = re.params[in_both_coef]
    v_b = fe.cov.loc[in_both_coef, in_both_coef]
    v_B = re.cov.loc[in_both_coef, in_both_coef]
    diff = b - B
    stat = float(diff.T @ la.inv(v_b - v_B) @ diff)
    d_f = len(diff)
    pval = stats.chi2.sf(stat, d_f)
    return stat, d_f, pval


In [14]:
chi2, df_n, pval = hausman(fe_results, re_results)
print(f"Hausman chi²: {chi2:.4f}")
print(f"Degrees of freedom: {df_n}")
print(f"p-value: {pval:.4f}")

if pval < 0.05:
    print("Conclusion: Reject H₀ and Use Fixed Effects")
else:
    print("Conclusion: Fail to reject H₀ so Random Effects acceptable")

Hausman chi²: -107.1807
Degrees of freedom: 20
p-value: 1.0000
Conclusion: Fail to reject H₀ so Random Effects acceptable


In [15]:
chi2, df_n, pval = hausman(fe_results_log, re_results_log)
print(f"Hausman chi²: {chi2:.4f}")
print(f"Degrees of freedom: {df_n}")
print(f"p-value: {pval:.4f}")

if pval < 0.05:
    print("Conclusion: Reject H₀ and Use Fixed Effects with log variable")
else:
    print("Conclusion: Fail to reject H₀ so Random Effects  with log variable acceptable")

Hausman chi²: 454.6653
Degrees of freedom: 20
p-value: 0.0000
Conclusion: Reject H₀ and Use Fixed Effects with log variable


## SO WE USE FIXED EFFECTS WITH LOG VARIABLE

In [16]:
from statsmodels.stats.stattools import durbin_watson

residuals_fe = fe_results_log.resids.dropna()
dw_stat_fe = durbin_watson(residuals_fe)
print(dw_stat_fe)

residuals_re = re_results_log.resids.dropna()
dw_stat_re = durbin_watson(residuals_re)
print(dw_stat_re)

0.4246889764344914
0.7483857693190781


## Clustered Model


In [17]:
fe_model = PanelOLS.from_formula(
    'Number_of_Beverages_Units_Sold ~ Number_of_Food_Service_Units_Sold + Discount_Availability_Food_Service + log_Discount_Amount_Food_Service + Discount_Availability_Bevareges + log_Discount_Amount_Beverages + log_Inventory_Turnover_Ratio_Food_Service + Inventory_Turnover_Ratio_ + 2022_Dummy + 2023_Dummy + February_Dummy + Murch_Dummy + April_Dummy + May_Dummy + June_Dummy + July_Dummy + August_Dummy + September_Dummy + October_Dummy + November_Dummy + December_Dummy + EntityEffects',
    data=df)
fe_clustered = fe_model.fit(cov_type="clustered", cluster_entity=True)
print(str(fe_clustered.summary).replace(
    "PanelOLS", "Fixed Effects (clustered SE)"))

                                Fixed Effects (clustered SE) Estimation Summary                                 
Dep. Variable:     Number_of_Beverages_Units_Sold   R-squared:                        0.6702
Estimator:                               Fixed Effects (clustered SE)   R-squared (Between):              0.9399
No. Observations:                            2470   R-squared (Within):               0.6702
Date:                            Sun, May 04 2025   R-squared (Overall):              0.9274
Time:                                    23:37:42   Log-likelihood                 -2.06e+04
Cov. Estimator:                         Clustered                                           
                                                    F-statistic:                      239.99
Entities:                                      88   P-value                           0.0000
Avg Obs:                                   28.068   Distribution:                 F(20,2362)
Min Obs:                      

In [18]:
residuals_fe = fe_clustered.resids.dropna()
dw_stat_fe = durbin_watson(residuals_fe)
print(dw_stat_fe)

0.7882081051471238


## First LAG

In [19]:
df_lagged = df.copy().reset_index()

df_lagged['lag_Number_of_Beverages'] = (
    df_lagged
    .groupby('Store_ID')['Number_of_Beverages_Units_Sold']
    .shift(1)
)
df_lagged = df_lagged.dropna(subset=['lag_Number_of_Beverages'])
df_lagged = df_lagged.set_index(['Store_ID', 'Data'])

formula_lag = (
    'Number_of_Beverages_Units_Sold ~ lag_Number_of_Beverages + '
    'Number_of_Food_Service_Units_Sold + Discount_Availability_Food_Service + '
    'log_Discount_Amount_Food_Service + Discount_Availability_Bevareges + '
    'log_Discount_Amount_Beverages + log_Inventory_Turnover_Ratio_Food_Service + '
    'Inventory_Turnover_Ratio_ + 2022_Dummy + 2023_Dummy + '
    'February_Dummy + Murch_Dummy + April_Dummy + May_Dummy + June_Dummy + '
    'July_Dummy + August_Dummy + September_Dummy + October_Dummy + '
    'November_Dummy + December_Dummy + EntityEffects'
)

fe_lag_model = PanelOLS.from_formula(formula_lag, data=df_lagged)
fe_lag_results = fe_lag_model.fit(cov_type="robust")

print(str(fe_lag_results.summary).replace(
    "PanelOLS", "Fixed Effects (robust SE)"))

                                Fixed Effects (robust SE) Estimation Summary                                 
Dep. Variable:     Number_of_Beverages_Units_Sold   R-squared:                        0.7583
Estimator:                               Fixed Effects (robust SE)   R-squared (Between):              0.8867
No. Observations:                            2382   R-squared (Within):               0.7583
Date:                            Sun, May 04 2025   R-squared (Overall):              0.8755
Time:                                    23:37:42   Log-likelihood                -1.945e+04
Cov. Estimator:                            Robust                                           
                                                    F-statistic:                      340.38
Entities:                                      83   P-value                           0.0000
Avg Obs:                                   28.699   Distribution:                 F(21,2278)
Min Obs:                            

In [20]:
residuals_fe = fe_lag_results.resids.dropna()
dw_stat_fe = durbin_watson(residuals_fe)
print(dw_stat_fe)

1.369915947655864


## Second LAG

In [ ]:
df_lagged = df.copy().reset_index()

df_lagged['lag_Number_of_Beverages'] = (
    df_lagged
    .groupby('Store_ID')['Number_of_Beverages_Units_Sold']
    .shift(1)
)
df_lagged['lag2_Number_of_Beverages'] = (
    df_lagged
    .groupby('Store_ID')['Number_of_Beverages_Units_Sold']
    .shift(2)
)

df_lagged = df_lagged.dropna(
    subset=['lag_Number_of_Beverages', 'lag2_Number_of_Beverages'])

df_lagged = df_lagged.set_index(['Store_ID', 'Data'])

formula_lag = (
    'Number_of_Beverages_Units_Sold ~ lag_Number_of_Beverages + lag2_Number_of_Beverages + '
    'Number_of_Food_Service_Units_Sold + Discount_Availability_Food_Service + '
    'log_Discount_Amount_Food_Service + Discount_Availability_Bevareges + '
    'log_Discount_Amount_Beverages + log_Inventory_Turnover_Ratio_Food_Service + '
    'Inventory_Turnover_Ratio_ + 2022_Dummy + 2023_Dummy + '
    'February_Dummy + Murch_Dummy + April_Dummy + May_Dummy + June_Dummy + '
    'July_Dummy + August_Dummy + September_Dummy + October_Dummy + '
    'November_Dummy + December_Dummy + EntityEffects'
)
fe_lag_model = PanelOLS.from_formula(formula_lag, data=df_lagged)
fe_lag_results = fe_lag_model.fit(cov_type="robust")

print(str(fe_lag_results.summary).replace(
    "PanelOLS", "Fixed Effects (robust SE)"))

                                Fixed Effects (robust SE) Estimation Summary                                 
Dep. Variable:     Number_of_Beverages_Units_Sold   R-squared:                        0.7595
Estimator:                               Fixed Effects (robust SE)   R-squared (Between):              0.8903
No. Observations:                            2299   R-squared (Within):               0.7595
Date:                            Mon, May 05 2025   R-squared (Overall):              0.8820
Time:                                    11:11:59   Log-likelihood                -1.874e+04
Cov. Estimator:                            Robust                                           
                                                    F-statistic:                      315.28
Entities:                                      81   P-value                           0.0000
Avg Obs:                                   28.383   Distribution:                 F(22,2196)
Min Obs:                            

In [25]:
residuals_fe = fe_lag_results.resids.dropna()
dw_stat_fe = durbin_watson(residuals_fe)
print(dw_stat_fe)

1.4901355690438223


## SO WE USE fe_lag_results model

In [26]:
from linearmodels.panel import PanelOLS
import numpy as np

def fe_model_selection(df, y_col, x_cols):
    current_vars = x_cols.copy()
    best_model = PanelOLS.from_formula(
        f"{y_col} ~ {' + '.join(current_vars)} + EntityEffects", data=df
    ).fit()

    best_llf = best_model.loglik
    best_k = len(best_model.params)
    best_n = best_model.nobs
    best_aic = 2 * best_k - 2 * best_llf

    print(f"Initial AIC: {best_aic:.2f}")

    while True:
        improved = False
        aic_scores = {}

        for var in current_vars:
            test_vars = [v for v in current_vars if v != var]
            formula = f"{y_col} ~ {' + '.join(test_vars)} + EntityEffects"
            model = PanelOLS.from_formula(formula, data=df).fit()
            k = len(model.params)
            llf = model.loglik
            aic = 2 * k - 2 * llf
            aic_scores[var] = aic

        best_var = min(aic_scores, key=aic_scores.get)
        min_aic = aic_scores[best_var]

        if min_aic < best_aic:
            print(
                f"Removed {best_var}: AIC improved from {best_aic:.2f} to {min_aic:.2f}")
            current_vars.remove(best_var)
            best_aic = min_aic
            improved = True
        if not improved:
            break

    final_formula = f"{y_col} ~ {' + '.join(current_vars)} + EntityEffects"
    final_model = PanelOLS.from_formula(
        final_formula, data=df).fit(cov_type="robust")
    print(f"Final AIC: {best_aic:.2f}")
    print(f"Remaining variables: {current_vars}")
    return final_model

In [27]:
independent_vars = [
    'lag_Number_of_Beverages',
    'Number_of_Food_Service_Units_Sold',
    'Discount_Availability_Food_Service',
    'log_Discount_Amount_Food_Service',
    'Discount_Availability_Bevareges',
    'log_Discount_Amount_Beverages',
    'log_Inventory_Turnover_Ratio_Food_Service',
    'Inventory_Turnover_Ratio_',
    '2022_Dummy', '2023_Dummy',
    'February_Dummy', 'Murch_Dummy', 'April_Dummy', 'May_Dummy',
    'June_Dummy', 'July_Dummy', 'August_Dummy', 'September_Dummy',
    'October_Dummy', 'November_Dummy', 'December_Dummy'
]

fe_aic_model = fe_model_selection(
    df_lagged, 'Number_of_Beverages_Units_Sold', independent_vars)

print(fe_aic_model.summary)

Initial AIC: 37566.25
Removed Discount_Availability_Food_Service: AIC improved from 37566.25 to 37564.25
Removed log_Inventory_Turnover_Ratio_Food_Service: AIC improved from 37564.25 to 37562.57
Removed October_Dummy: AIC improved from 37562.57 to 37560.91
Removed log_Discount_Amount_Beverages: AIC improved from 37560.91 to 37559.26
Final AIC: 37559.26
Remaining variables: ['lag_Number_of_Beverages', 'Number_of_Food_Service_Units_Sold', 'log_Discount_Amount_Food_Service', 'Discount_Availability_Bevareges', 'Inventory_Turnover_Ratio_', '2022_Dummy', '2023_Dummy', 'February_Dummy', 'Murch_Dummy', 'April_Dummy', 'May_Dummy', 'June_Dummy', 'July_Dummy', 'August_Dummy', 'September_Dummy', 'November_Dummy', 'December_Dummy']
                                PanelOLS Estimation Summary                                 
Dep. Variable:     Number_of_Beverages_Units_Sold   R-squared:                        0.7548
Estimator:                               PanelOLS   R-squared (Between):             

## Final model

In [28]:
final_vars = [
    'lag_Number_of_Beverages',
    'Number_of_Food_Service_Units_Sold',
    'log_Discount_Amount_Food_Service',
    'Discount_Availability_Bevareges',
    'Inventory_Turnover_Ratio_',
    '2022_Dummy', '2023_Dummy',
    'February_Dummy', 'Murch_Dummy', 'April_Dummy', 'May_Dummy',
    'June_Dummy', 'July_Dummy', 'August_Dummy', 'September_Dummy',
    'October_Dummy', 'November_Dummy', 'December_Dummy'
]
final_formula = (
    'Number_of_Beverages_Units_Sold ~ ' +
    ' + '.join(final_vars) +
    ' + EntityEffects'
)

final_model = PanelOLS.from_formula(final_formula, data=df_lagged)
final_results = final_model.fit(cov_type='robust')
print(final_results.summary)

                                PanelOLS Estimation Summary                                 
Dep. Variable:     Number_of_Beverages_Units_Sold   R-squared:                        0.7549
Estimator:                               PanelOLS   R-squared (Between):              0.8730
No. Observations:                            2299   R-squared (Within):               0.7549
Date:                            Mon, May 05 2025   R-squared (Overall):              0.8654
Time:                                    11:18:46   Log-likelihood                -1.876e+04
Cov. Estimator:                            Robust                                           
                                                    F-statistic:                      376.35
Entities:                                      81   P-value                           0.0000
Avg Obs:                                   28.383   Distribution:                 F(18,2200)
Min Obs:                                   2.0000                     